In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Imports

In [64]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
from datasets import Dataset
from pathlib import Path
import os, sys
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
import torch
import argparse


import h5py
import hdf5plugin
import torch
import torch.nn
import torch.nn.functional as F
import torchaudio
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoModel, Wav2Vec2FeatureExtractor
import uuid
# from acestep.pipeline_ace_step import ACEStepPipeline
# from acestep.text2music_dataset import Text2MusicDatase

In [2]:
PROJECT_DIR="/home/rileywilliams/soundverse/ACE-step-lowvram"

# Initializationm

In [61]:
AUDIO_SRC_DIR="/mnt/disks/audio-ai-research-speech-data/rjw_audio/Riley_Full_Catalogue_PROCESSED/"
NEW_AUDIO_SRC_DIR="/mnt/disks/lora_training_data/rjw_all_preprocessed/"
annotations_manifest_file=os.path.join(
    PROJECT_DIR,
    *['experimental/rjw/', 'raw_data_rjw.csv']
)
DATASET_DIR="/mnt/disks/audio-ai-research-speech-data/ace_step_datasets/rjw_all_LoRa_data"
PROCESSED_DATASET_DIR="$DATASET_DIR/processed"

In [84]:
def format_example_json_to_prompt(
    example, split_phrases=True, combine_for_tags=False, prepended_trigger_word=None
):
    analysis = example.get("analysis")
    if analysis is not None:
        genres = example["analysis"]["genre"][:3]  # take top 3 genres
        tempo = example["analysis"]["tempo"]
    else:
        genres, tempo = [""], ""
    prompt = (
        example["expanded"].rstrip(".")
        + f'. In the genre of {(" and ").join(genres)}. '
        + tempo.capitalize()
    )
    if prepended_trigger_word is not None:
        prompt = ('. ').join([prepended_trigger_word, prompt])
    if split_phrases:
        prompt = prompt.split(". ")
    if combine_for_tags:
        prepended_trigger_word
        return ','.join(prompt).strip()
    return prompt

In [59]:
 # os.listdir(DATASET_DIR)#[0]
 # os.listdir(DATASET_DIR)
anno_df = pd.read_csv(annotations_manifest_file)
single_record = anno_df.to_dict(orient="records")[0]
formatted_response = eval(single_record["formatted_response"])

In [75]:
my_uuid = uuid.uuid4().hex
print(f'my_uuid: {my_uuid}')

my_uuid: 4abe0dc877704be489c89dc07927921f


'4abe0dc877704be489c89dc07927921f'

In [87]:
for _index, _row in anno_df.iterrows():
    filepath = _row['filepath']
    src_basename = os.path.basename(filepath)
    src_basename, src_ext = os.path.splitext(src_basename)
    prompt_dst_filename = os.path.join(NEW_AUDIO_SRC_DIR, src_basename + '_prompt.txt')
    lyrics_dst_filename = os.path.join(NEW_AUDIO_SRC_DIR, src_basename + '_lyrics.txt')
    
    formatted_response = eval(_row["formatted_response"])

    text_to_save = format_example_json_to_prompt(
        example=formatted_response,
        combine_for_tags=True,
        prepended_trigger_word=my_uuid
    )
    # print(text_to_save)
    with open(prompt_dst_filename, "w", encoding="utf-8", newline="\n") as f:
        f.write(text_to_save)

    lyrics = "[instrumental]"
    with open(lyrics_dst_filename, "w", encoding="utf-8", newline="\n") as f:
        f.write(lyrics)

In [49]:
pp.pprint(list(single_record.keys()))
# pp.pprint(single_record["formatted_response"])


[   'filepath',
    'raw_response',
    'execution_time',
    'formatted_response',
    'analysis',
    'descriptive',
    'expanded',
    'simplified',
    'synonyms',
    'use_cases']


In [38]:

pp.pprint(single_record)

{   'analysis': "{'genre': ['electronic', 'dance', 'ambient'], 'instruments': "
                "['synthesizer', 'drummachine', 'piano', 'bass'], 'tempo': "
                "'118 BPM', 'mood': ['energetic', 'futuristic', 'hypnotic']}",
    'descriptive': 'The audio features a pulsating electronic beat with '
                   'synthesized sounds creating an immersive, futuristic '
                   'atmosphere.',
    'execution_time': 644.9457767250024,
    'expanded': 'Dance music, ambient music',
    'filepath': '/mnt/disks/audio-ai-research-speech-data/rjw_audio/Riley_Full_Catalogue_PROCESSED/MERIDIAN_Lessons_in_Falling_MASTER_2023_07_04_00_16bit__000 '
                'copy.wav',
    'formatted_response': "{'simplified': 'Electronic music', 'expanded': "
                          "'Dance music, ambient music', 'descriptive': 'The "
                          'audio features a pulsating electronic beat with '
                          'synthesized sounds creating an immersive, '
  